In [24]:
import subprocess
from pathlib import Path

materials = "h-CrO2"
filename = f"ptest/{materials}_1.lmp"
Path(filename).unlink(missing_ok=True)


atomsk_command = f"atomsk tribo_2D/cif/{materials}.cif -orthogonal-cell -duplicate 2 1 1 -ow {filename} "
subprocess.run(atomsk_command, shell=True, check=True)

 ___________________________________________________
|              ___________                          |
|     o---o    A T O M S K                          |
|    o---o|    Version Beta 0.13.1                  |
|    |   |o    (C) 2010 Pierre Hirel                |
|    o---o     https://atomsk.univ-lille.fr         |
|___________________________________________________|
*** Working out of office hours? You should sleep sometimes. :-)
>>> Opening the input file: tribo_2D/cif/h-CrO2.cif
..> Applying symmetry operations...
..> Input file was read successfully (3 atoms).
>>> Converting system into an orthorhombic cell...
..> A suitable cell was found, now filling it with atoms...
    (estimated new number of atoms: 7)
..> Cell is now orthorhombic (6 atoms).
>>> Duplicating the system: 2 x 1 x 1
..> System was successfully duplicated (12 atoms).
>>> Writing output file(s) (12 atoms):
..> Successfully wrote LAMMPS data file: ptest/h-CrO2_1.lmp
\o/ Program terminated successfully!
    Tot

CompletedProcess(args='atomsk tribo_2D/cif/h-CrO2.cif -orthogonal-cell -duplicate 2 1 1 -ow ptest/h-CrO2_1.lmp ', returncode=0)

In [ ]:
import os
import subprocess
import numpy as np
import re
from pathlib import Path
from ase.io import read

x=100
y=100

# Now, you can import tribo_2D
from tribo_2D import tools, settings
with open("material_list.txt", "r") as f:
    materials = [line.strip() for line in f]

# atomsk_command = f"atomsk Program/tribo_2D/cif/{material}.cif -duplicate 2 2 1 -orthogonal-cell a.lmp"
for mat in materials:
    filename = f"ptest/{mat}_1.lmp"
    
    pot = tools.count_elemtypes(f"tribo_2D/Potentials/sw_lammps/{mat}.sw")
    cif = tools.cifread(f"tribo_2D/cif/{mat}.cif")

    multiples = {}

    for element, cif_count in cif["elem_comp"].items():
        potential_count = pot.get(element, 0)  # Get count from potential file or default to 0
        if cif_count > 0 and potential_count != 1:  # Avoid division by zero
            multiples[element] = potential_count / cif_count
        else:
            multiples[element] = None  # If the CIF count is zero (which shouldn't happen), mark as None

    first_multiple = next(iter(multiples.values()))  # Get the first multiple

    for multiple in multiples.values():
        if multiple != first_multiple:  
            if multiple == None and first_multiple == 1 or multiple == 1 and first_multiple== None:
                first_multiple=1
            else:
                raise ValueError("multiples must be the same")

    typecount = 0
    for atomcount in pot.values():
        typecount += atomcount
 
    Path(filename).unlink(missing_ok=True)

    if first_multiple is None:
        atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -ow {filename} -v 0"

    elif isinstance(first_multiple, float) and not first_multiple.is_integer():
        raise ValueError('potential file or cif file is not formatted properly')
    
    else:
        if first_multiple ==1:
            atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -ow {filename} -v 0"
        else:
            atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -orthogonal-cell -ow {filename} -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            _ = tools.num_atoms_lmp(filename)

            atomsk_command = f"echo n | atomsk {filename} -orthogonal-cell -ow lmp -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            atoms = read(filename, format="lammps-data")
            # Get number of atoms
            natoms = len(atoms)

            if typecount % natoms == 0:
                for i in range(int(np.sqrt(typecount/natoms))+1, 0, -1):
                    if typecount/natoms % i == 0:
                        a = int(i)
                        b = int(typecount/natoms / i)
                        break

            atomsk_command = f"echo n | atomsk {filename} -duplicate {a} {b} 1 -ow lmp -v 0"
            print(a,b)
    
    print(atomsk_command)
    subprocess.run(atomsk_command, shell=True, check=True)

    elem2D = tools.num_atoms_lmp(filename)

    if first_multiple == None or first_multiple == 1:
        elem2D = tools.num_atoms_lmp(filename)
        atomsk_command = f"atomsk {filename} -orthogonal-cell -ow lmp"
        subprocess.run(atomsk_command, shell=True, check=True)


dim = tools.get_model_dimensions(filename)
duplicate_a = round(x / dim['xhi'])
duplicate_b = round(y / dim['yhi'])

atomsk_command = f"atomsk {filename} -duplicate {duplicate_a} {duplicate_b} 1 -ow lmp -v 0"
subprocess.run(atomsk_command, shell=True, check=True)

    # atomsk_command = f"atomsk {filename} -duplicate 2 2 1 -ow lmp"
    # subprocess.run(atomsk_command, shell=True, check=True)

/!\ WARNING: cell is already orthorhombic, skipping.
6
2 1
echo n | atomsk ptest/h-CrO2_1.lmp -duplicate 2 1 1 -ow lmp -v 0


CompletedProcess(args='atomsk ptest/h-CrO2_1.lmp -duplicate 19 22 1 -ow lmp -v 0', returncode=0)

In [3]:
def num_atoms_lmp(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    modified_lines = []
    masses_section = False
    atoms_section = False
    a = 1 

    atom_types = {}
    elem = {}
    for i, line in enumerate(lines):
        stripped_line = line.strip()

        if line.strip() == 'Masses':
            masses_section = True
            continue  
        
        if masses_section:
            if 'Atoms' in line:
                break
            
            parts = line.split()
            if len(parts) < 2:
                continue

            atom_type_id = int(parts[0])  
            mass = float(parts[1])  
            if '#' in line:
                atom_type_name = line.split('#')[-1].strip()
                lines[i]= ''
            else:
                atom_type_name = f'Unknown_{atom_type_id}'  
            atom_types[atom_type_id] = (atom_type_name, mass)
  
   
    modified_lines = set() 

    for i in range(1,len(atom_types)+1):
        atoms_section = False
        for l, line in enumerate(lines):
            stripped_line = line.strip()

            if 'Atoms' in line:
                atoms_section = True
                continue
            if atoms_section and stripped_line and l not in modified_lines:
                    parts = stripped_line.split()
                    if parts[1] == str(i):
                        parts[1] = str(a)  # Update atom type
                        lines[l]= '  '.join(parts) + '\n'
                        modified_lines.add(l)
                        elem[a] = atom_types[i]
                        a += 1  # Increment for next line
                        continue

    masses_section = False
    for i, line in enumerate(lines):
        if re.match(r'^\s*\d+\s+atom types\s*$', line.strip()):
            print(len(elem))
            lines[i]= f"  {len(elem)}  atom types\n"
            continue

        if line.strip() == 'Masses':
            masses_section = True
            continue
        
        if masses_section:
            for l in range(1,len(elem)+1):
                lines[i] += f"{l} {elem[l][1]}  #{elem[l][0]}\n"
            break
        
    # Save modified file
    with open(filename, 'w') as file:
        file.writelines(lines)
    
    return elem

In [2]:
import re

def process_h(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()


    masses_section = False
    atoms_section = False
    
    atom_types = {}
    for i, line in enumerate(lines):

        if line.strip() == 'Masses':
            masses_section = True
            continue  
        
        if masses_section:
            if 'Atoms' in line:
                break
            
            parts = line.split()
            if len(parts) < 2:
                continue

            atom_type_id = int(parts[0])  
            mass = float(parts[1])  
            if '#' in line:
                atom_type_name = line.split('#')[-1].strip()
                lines[i]= ''
            else:
                atom_type_name = f'Unknown_{atom_type_id}'  
            atom_types[atom_type_id] = (atom_type_name, mass)

    # Update atom types
    modified_lines = set() 
    mod_lines = {}
    pre_elem = {}
    elem = {}
    # for i in range(1,len(atom_types)+1):
    for i in range(1,len(atom_types)+1):
        atoms_section = False
        t = i
        for l, line in enumerate(lines):
            stripped_line = line.strip()
            
            if 'Atoms' in line:
                atoms_section = True
                continue
            
            if atoms_section and stripped_line and l not in modified_lines:
                parts = stripped_line.split()

                if parts[1] == str(i):
                    parts[1] = parts[0] = str(t)
                    lines[l]= ''
                    mod_lines[t] = '  '.join(parts) + '\n'
                    modified_lines.add(l)
                    pre_elem[t] = atom_types[i]
                    t+= len(atom_types)

    modified_lines = set() 

    a = 1
    unique = list(set(val[0] for val in atom_types.values()))
    atom_lines = {}
    for i in unique:

        atoms_section = False
        for l in range(1,len(mod_lines)+1):
        
            stripped_line = mod_lines[l].strip()
            
            parts = stripped_line.split()

            if pre_elem[int(parts[1])][0] == i:
                elem[a] = pre_elem[int(parts[1])]
                parts[1] = str(a)
                atom_lines[a] = '  '.join(parts) + '\n'
                a+= 1
              


    
    # Update atom types line
    masses_section = False
    for i, line in enumerate(lines):
        if re.match(r'^\s*\d+\s+atom types\s*$', line.strip()):
            lines[i]= f"  {len(elem)}  atom types\n"
            continue

        if line.strip() == 'Masses':
            masses_section = True
            continue
        
        if masses_section:
            for l in range(1,len(elem)+1):
                lines[i] += f"{l} {elem[l][1]}  #{elem[l][0]}\n"
            break

    # Write updated file
    with open(filename, 'w') as f:
        f.writelines(lines)
    with open(filename, 'a') as f:  # 'a' means append mode
        for line in atom_lines.values():
            f.write(line)

# filename = f"ptest/{mat}_1.lmp"
# process_h(filename)

In [ ]:
import os
import subprocess
import numpy as np
import re
from pathlib import Path
from ase.io import read

x=100
y=100

# Now, you can import tribo_2D
from tribo_2D import tools, settings
with open("material_list.txt", "r") as f:
    materials = [line.strip() for line in f]


for mat in materials:
    filename = f"ptest/{mat}_1.lmp"
    
    pot = tools.count_elemtypes(f"tribo_2D/Potentials/sw_lammps/{mat}.sw")
    cif = tools.cifread(f"tribo_2D/cif/{mat}.cif")

    multiples = {}

    for element, cif_count in cif["elem_comp"].items():
        potential_count = pot.get(element, 0)  # Get count from potential file or default to 0
        if cif_count > 0 and potential_count != 1:  # Avoid division by zero
            multiples[element] = potential_count / cif_count
        else:
            multiples[element] = None  # If the CIF count is zero (which shouldn't happen), mark as None

    first_multiple = next(iter(multiples.values()))  # Get the first multiple

    for multiple in multiples.values():
        if multiple != first_multiple:  
            if multiple == None and first_multiple == 1 or multiple == 1 and first_multiple== None:
                first_multiple=1
            else:
                raise ValueError("multiples must be the same")

    typecount = 0
    for atomcount in pot.values():
        typecount += atomcount
 
    Path(filename).unlink(missing_ok=True)

    # atomsk_command = f"atomsk Program/tribo_2D/cif/{material}.cif -duplicate 2 2 1 -orthogonal-cell a.lmp"
    filename = f"ptest/{mat}_1.lmp"

    pot = tools.count_elemtypes(f"tribo_2D/Potentials/sw_lammps/{mat}.sw")
    cif = tools.cifread(f"tribo_2D/cif/{mat}.cif")
    multiples = {}
    for element, cif_count in cif["elem_comp"].items():
        potential_count = pot.get(element, 0)  # Get count from potential file or default to 0
        if cif_count > 0 and potential_count != 1:  # Avoid division by zero
            multiples[element] = potential_count / cif_count
        else:
            multiples[element] = None  # If the CIF count is zero (which shouldn't happen), mark as None


    first_multiple = next(iter(multiples.values()))  # Get the first multiple
    for multiple in multiples.values():
        if multiple != first_multiple:  
            if multiple == None and first_multiple == 1 or multiple == 1 and first_multiple== None:
                first_multiple=1
            else:
                raise ValueError("multiples must be the same")

    typecount = 0
    for atomcount in pot.values():
        typecount += atomcount
    print(mat)
    if first_multiple is None:
        atomsk_command = f"echo n | atomsk ./tribo_2D/cif/{mat}.cif -ow {filename} -v 0"
        subprocess.run(atomsk_command, shell=True, check=True)
        atomsk_command = f"atomsk {filename} -orthogonal-cell -ow lmp -v 0"
        subprocess.run(atomsk_command, shell=True, check=True)
    else:
        if first_multiple ==1:
            atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -ow {filename} -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            num_atoms_lmp(filename)
            atomsk_command = f"atomsk {filename} -orthogonal-cell -ow lmp -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
        else:
            atomsk_command = f"echo n | atomsk tribo_2D/cif/{mat}.cif -ow {filename} -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            num_atoms_lmp(filename)
            atomsk_command = f"echo n | atomsk {filename} -orthogonal-cell -ow lmp -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            atoms = read(filename, format="lammps-data")
            # Get number of atoms
            natoms = len(atoms)
            if typecount % natoms == 0:
                for i in range(int(np.sqrt(typecount/natoms))+1, 0, -1):
                    if typecount/natoms % i == 0:
                        a = int(i)
                        b = int(typecount/natoms / i)
                        break        
            atomsk_command = f"echo n | atomsk {filename} -duplicate {a} {b} 1 -ow lmp -v 0"
            subprocess.run(atomsk_command, shell=True, check=True)
            tools.process_h(pot,filename)

    atomsk_command = f"atomsk {filename} -duplicate 2 4 1 -ow lmp -v 0"
    subprocess.run(atomsk_command, shell=True, check=True)

p-GeSe
4
/!\ WARNING: cell is already orthorhombic, skipping.


In [95]:
dim = tools.get_model_dimensions(filename)
duplicate_a = round(x / dim['xhi'])
duplicate_b = round(y / dim['yhi'])

atomsk_command = f"atomsk {filename} -duplicate {duplicate_a} {duplicate_b} 1 -ow lmp -v 0"
subprocess.run(atomsk_command, shell=True, check=True)

    # atomsk_command = f"atomsk {filename} -duplicate 2 2 1 -ow lmp"
    # subprocess.run(atomsk_command, shell=True, check=True)

CompletedProcess(args='atomsk ptest/h-CrO2_1.lmp -duplicate 19 22 1 -ow lmp -v 0', returncode=0)

In [18]:
from ase.io import read, write
import subprocess
import os

with open("material_list.txt", "r") as f:
    materials = [line.strip() for line in f]

materials = ["p-GeTe"]
for material in materials:

    compile_process = subprocess.run(["gfortran", "-g", f"f90lammps/xyz/{material.lower()}.f90"])
    run_process = subprocess.run(["./a.out"])
    # ovito = subprocess.run(["ovito", "xyz.xsf"])
    os.rename("lammps.dat", f"f90lammps/{material}.lmp")


 # orientation, ntot = a          64
 # system = sheet; lenxyz(1:3) =   17.503999710083008        16.951999664306641        5.0000000000000000     
 # mdbox(1:3) =    17.503999710083008        16.951999664306641        105.00000000000000     
 # nx, ny, nz =           2           4           1
 Job done, Sir!
